In [19]:
# Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc

import os
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests

In [20]:
# Configure Chrome Webdriver

chrome_install = ChromeDriverManager().install()

In [21]:
# Initialize Chrome WebDriver
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
options.add_argument(
    "--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 17_0 like Mac OS X) "
    "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1"
)
browser = webdriver.Chrome(options=options)

In [22]:
# Setup search parameters
city = "montreal"
city_link = "montréal"
product = "iphone"
min_price = 0
max_price = 5000
days_listed = 60

In [23]:
# Setup base URL
url = f'https://www.facebook.com/marketplace/{city}/search?query={product}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_listed}&exact=false'

# Visit website
browser.get(url)

# Close login pop-up
try:
    close_button = browser.find_element(By.XPATH, "//div[@aria-label='Close']")
    close_button.click()
    print("Login prompt closed successfully...")
except:
    print("Login prompt not closed correctly...")
    pass

Login prompt not closed correctly...


In [24]:
# Scroll down to load all results
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
    print("Scrolled..")

print("Finished scrolling, all results loaded...")


Scrolled..
Finished scrolling, all results loaded...


In [25]:
# Retrieve the HTML
html = browser.page_source

soup = BeautifulSoup(html, 'html.parser')

browser.close()

In [ ]:
# Find link elements
links = soup.find_all('a')

# Filter on product keyword only
product_links = [
    link for link in links
    if product.lower() in link.text.lower()
    #if city_link.lower() in link.text.lower()
]

product_data = []

for product_link in product_links:
    url = product_link.get('href')
    if not url:
        continue
    text = '\n'.join(product_link.stripped_strings)
    product_data.append({'url': url, 'text': text})

print(len(product_links), "links matched")
print(len(product_data), "items collected")

product_data

47 links matched
47 items collected


[{'url': '/marketplace/item/616854888161430/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$1,600 iPhone 17 Pro Montréal, QC'},
 {'url': '/marketplace/item/1381661193728377/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$800 iPhone 15 plus 128 gb Montréal, QC'},
 {'url': '/marketplace/item/2265568590608152/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$250 iPhone 13 64go St-Jean-sur-Richelieu, QC'},
 {'url': '/marketplace/item/2693158294357331/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$200 iPhone 11 neuf (utilisé 1 semaine) Montréal, QC'},
 {'url': '/marketplace/item/1321058079349773/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$720 iPhone 15 Montréal, QC'},
 {'url': '/marketplace/item/601679823004437/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$399 apple i

In [27]:
# Create an empty list to store product data
extracted_data = []

for item in product_data:
    lines = item['text'].split('\n')

    # Regular expression to find numeric values
    numeric_pattern = re.compile('\d[\d,.]*')
    
    
    # Extracting prices
    # Iterate through lines to find the first line with numbers
    for line in lines:
        match = numeric_pattern.search(line)
        if match:    
            # Extract the first numeric value found
            price_str = match.group()
            # Convert price to float (handle commas)
            price = float(price_str.replace(',',''))
            break
            
    if price:
        print(f"Price extracted: {price}")
    else:
        print("price not found")

    # Extract title
    title = lines[-2]

    # Extract location
    location = lines[-1]

    # Add extracted data to a list of dictionaries
    extracted_data.append({
        'title': title,
        'price': price,
        'location': location,
        'url': re.sub(r'\?.*', '', item['url'])
    })

Price extracted: 1600.0


<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
/var/folders/xn/w_xnr1cn1jq5m_6rgnybnsc40000gn/T/ipykernel_17521/1258939502.py:8: SyntaxWarning: invalid escape sequence '\d'
  numeric_pattern = re.compile('\d[\d,.]*')


IndexError: list index out of range

In [ ]:
extracted_data